# Entrega final

**(Se requiere el notebook "Análisis de sueldos" resuelto)**

En base a los resultados obtenidos en *"Análisis de sueldos"* confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | -----------------5%--------10%----------------40%
```

In [1]:
#Me armo el diccionario con los datos de la encuesta

#Nombro los csv

file_name_1 = "2021.2 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.csv"
file_name_2 = "2021.2 - sysarmy - Encuesta de remuneración salarial Latam - Latam.csv"

#Abro los archivos y guardo la info en una lista
data_arg_temp = []
data_latam_temp = []

import csv
with open(file_name_1, 'r', encoding = 'utf8') as datos_arg:
    reader = csv.reader(datos_arg)
    for row in reader:
        data_arg_temp.append(row)
        
with open(file_name_2, 'r', encoding = 'utf8') as datos_latam:
    reader = csv.reader(datos_latam)
    for row in reader:
        data_latam_temp.append(row)

# Borro las primeras filas de los datos, que no tienen la info de las respuestas a la encuesta
del data_arg_temp[:9]
del data_latam_temp[:11]

# Acomodo las columnas de ciudad-pais en LATAM
for case in data_latam_temp:
    i = 18
    while i <= 18 and i > 0:
        if len(case[i]) == 0:
            case.pop(i)
        i -= 1

# Agrego el pais en los datos de Argentina
for i in range(0,len(data_arg_temp)):
    if i==0:
        data_arg_temp[i].insert(0,"En que pais trabajas")
    else:
        data_arg_temp[i].insert(0,"Argentina")

#Uno las dos tablas en una
data_total_temp = data_arg_temp + data_latam_temp

#Lo guardo en un solo archivo
 
with open('Data.csv', 'w', encoding = 'utf8') as datos: 
    write = csv.writer(datos) 
    write.writerow(data_total_temp[0])
    write.writerows(data_total_temp[1:])  

#Armo diccionario de respuestas y cargo la info
data = []
with open('Data.csv', encoding = 'utf8') as datos:
    reader = csv.DictReader(datos)
    for row in reader:
        data.append(row)

#Cierro los archivos
datos_arg.close()
datos_latam.close()
datos.close()

In [2]:
#Creo una funcion para armar los diccionarios con el porcentaje de participacion 
def armado_dict_participacion(pregunta):
    
    #Guardo las respuestas en un set, para que me queden todas las respuestas unicas que hubo a cierta pregunta
    respuestas_unicas = set()
    for i in range(0,len(data)):
        if len(data[i][pregunta])>0:
            respuestas_unicas.add(data[i][pregunta])

    #Armo el diccionario de respuestas unicas y participantes
    participantes_por_respuesta = []
    for i in range(0,len(respuestas_unicas)):
        participantes_por_respuesta.append(0)
    respuestas_unicas = list(respuestas_unicas)
    dic_respuestas = dict(zip(respuestas_unicas, participantes_por_respuesta))

    cant_total_respuestas = 0
    #Cuento la cantidad de participantes que contestaron cada respuesta
    for i in range(0,len(data)):
        for respuesta in respuestas_unicas:
            if data[i][pregunta] == respuesta:
                dic_respuestas[respuesta] += 1
                cant_total_respuestas += 1

    # Armo diccionario para guardar los porcentajes de participacion por respuesta
    dic_participacion = dic_respuestas

    # Calculo porcentaje de participacion por respuesta
    for respuesta in respuestas_unicas:
         dic_participacion[respuesta]=round(dic_respuestas[respuesta]/cant_total_respuestas*100,2)

    return dic_participacion, respuestas_unicas

In [3]:
# Creo una funcion para ordenar diccionarios
def ordenar_dict(diccionario):
    valores_ordenados = sorted(diccionario.values(), reverse =True)
    diccionario_ordenado = {}

    for valor in valores_ordenados:
        for key in diccionario.keys():
            if diccionario[key] == valor:
                   diccionario_ordenado[key] = diccionario[key]
                    
    return diccionario_ordenado

# Creo una funcion para acomodar el formato del porcentaje

def formato_porcentaje(diccionario):
    for i in diccionario:
        diccionario[i] = f'{diccionario[i]}%'
    
    return diccionario

In [4]:
#Acomodo salarios

#Busco cuales son los paises que habia en la encuesta

paises = set()
for i in range(0, len(data)):
    paises.add(data[i]['En que pais trabajas'])

paises = list(paises)

#Me creo un diccionario con las cotizaciones por pais, que saque de Google

tipo_de_cambio = {'Uruguay': 44.57,
 'Costa Rica': 640.48,
 'Puerto Rico': 1,
 'Paraguay': 7055.63,
 'Argentina': 104.77,
 'Colombia': 3963.01,
 'Venezuela': 4.58,
 'Ecuador': 1,
 'Nicaragua': 35.49,
 'El Salvador': 8.76,
 'México': 20.73,
 'Perú': 3.85,
 'Bolivia': 6.89,
 'Guatemala': 7.70,
 'Chile': 805.25,
 'Honduras': 24.65,
 'Cuba': 24.02,
 'Panamá': 1,
 'República Dominicana': 50.7}

#Arreglo formatos de salario
import re
for i in range(0, len(data)):
    data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] = data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'].replace('$','')
    data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] = re.sub("[^0-9,.]","",data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'])
    data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] = data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'].replace(" ","")
    if len(data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'])>3:
        if data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'][-2] == '.':
            data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'][-2].replace('.','-')
        if data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'][-3] == '.':
            data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'][-3].replace('.','-')
    data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] = data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'].replace(".","")
    data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] = data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'].replace(",",".")
    if data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] == '.':
        data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] = ''

#Convierto el salario a USD y lo agrego como un nuevo key-value en el diccionario data
for i in range(0,len(data)):
    for pais in paises:
        if data[i]['En que pais trabajas'] == pais:
            if data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'] != "":
                salario_normalizado = round(float(data[i]['Salario mensual o retiro BRUTO (en tu moneda local)'])/tipo_de_cambio[pais],0)
            else: 
                salario_normalizado = ""
            data[i]['Salario en USD']=salario_normalizado
    

In [5]:
# Me creo una funcion para calcular la mediana del salario

def mediana_salario_por_preg(respuestas, pregunta):
    mediana_salario = {}
    for elemento in respuestas:
        mediana_salario[elemento] = 0
    for elemento in respuestas:
        salarios = []
        salarios_ordenado = []
        for i in range(0, len(data)):
            if data[i]['Salario en USD'] != "":
                if data[i][pregunta] == elemento:
                    salarios.append(data[i]['Salario en USD'])
        salarios_ordenado = sorted(salarios)
        if len(salarios)>1:
            if len(salarios)%2 != 0:
                indice_mediana = int(round(len(salarios)/2,0) - 1)
                mediana = salarios_ordenado[indice_mediana]
            else: 
                indice_1 = int(len(salarios)/2 - 1)
                indice_2 = int(len(salarios)/2)
                mediana = (salarios_ordenado[indice_1] + salarios_ordenado[indice_2])/2
        elif len(salarios) == 1:
            mediana = salarios_ordenado[0]
        else:
            mediana = 0
        mediana_salario[elemento] = mediana
        
    return ordenar_dict(mediana_salario)

In [6]:
# Me defino una funcion para graficar los resultados de participacion

def graficar(diccionario, porcentaje_corte, dicc_mediana_salario):
    largos = []
    for elem in diccionario.keys():
        if float(diccionario[elem][:-1]) >= porcentaje_corte:
            largos.append(len(elem))
    mas_largo = max(largos)

    porcentaje_acumulado = 0
    porcentaje_acum_real = 0

    for elem in diccionario.keys():
        if float(diccionario[elem][:-1]) >= porcentaje_corte:
            print(elem, end="")
            for i in range(0, mas_largo - len(elem) + 1):
                print(" ", end="")
            for i in range(0, round(float(diccionario[elem][:-1]))):
                print('-', end="")
            print(' ', end="")
            print(f'{round(float(diccionario[elem][:-1]),2)}%', end="")
            porcentaje_acumulado += round(float(diccionario[elem][:-1]))
            porcentaje_acum_real += round(float(diccionario[elem][:-1]),2)
            print(f'  {dicc_mediana_salario[elem]} USD')
    if porcentaje_acum_real < 99.99:
        print('Otros', end="")
        for i in range(0, mas_largo - len('Otros') + 1):
            print(" ", end="")
        for i in range(0, 100 - porcentaje_acumulado):
            print('-', end="")
        print(' ', end="")
        print(f'{round(100 - porcentaje_acum_real,2)}%')

In [7]:
# Participacion por rol

# Armo diccionario de roles y porcentaje de respuestas x rol
dic_participacion_por_rol, roles = armado_dict_participacion("Trabajo de")

# Ordeno el diccionario
sorted_dic_participacion_por_rol = ordenar_dict(dic_participacion_por_rol)

# Acomodo el formato de porcentaje
sorted_dic_participacion_por_rol = formato_porcentaje(sorted_dic_participacion_por_rol)

# Calculo la mediana de salario por rol
mediana_salario_por_rol = mediana_salario_por_preg(roles,"Trabajo de")

# Identifico el rol mejor pago
for i in mediana_salario_por_rol:
    if mediana_salario_por_rol[i] == max(mediana_salario_por_rol.values()):
        rol_mejor_pago = i
    
# Identifico el rol menor pago
mediana_salario_mayor_a_cero = set(mediana_salario_por_rol.values())
mediana_salario_mayor_a_cero.discard(0)
for i in mediana_salario_por_rol:
    if mediana_salario_por_rol[i] == min(mediana_salario_mayor_a_cero):
        rol_peor_pago = i


In [8]:
# Nivel de educacion vs Sueldos
#Armo diccionario de nivel educativo y cant de respuestas por nivel educativo
dic_participacion_por_nivel, nivel_educativo = armado_dict_participacion("Nivel de estudios alcanzado")

#Ordeno el diccionario
sorted_dic_participacion_por_nivel = ordenar_dict(dic_participacion_por_nivel)

#Acomodo el formato de porcentaje
sorted_dic_participacion_por_nivel = formato_porcentaje(sorted_dic_participacion_por_nivel)


#Calculo la mediana de salario
mediana_salario_por_nivel = mediana_salario_por_preg(nivel_educativo,"Nivel de estudios alcanzado")

#Ahora para discriminar por estado

estados = {'Completado': 0, 'Incompleto': 0, 'En curso': 0}
estados_indice = [['Completado', 0], ['Incompleto', 1], ['En curso', 2]]
estados_lista = ['Completado', 'Incompleto', 'En curso']
lista_salida = []

for nivel in nivel_educativo:
    lista_estados = [0, 0, 0]
    for i in range(0,len(data)):
        for estado in estados_indice:
            if data[i]["Nivel de estudios alcanzado"] == nivel and data[i]["Estado"] == estado[0]:
                lista_estados[estado[1]] += 1
    total_nivel = sum(lista_estados)
    porcentajes = []
    for x in range(0, len(lista_estados)):
        porcentajes.append(str(round(lista_estados[x]*100/total_nivel, 2)) + '%')
    dict_salida = dict(zip(estados_lista, porcentajes))
    dic_estado_educacion = {nivel: dict_salida}
    lista_salida.append(dic_estado_educacion)


In [9]:
#Genero vs Sueldos

#Acomodo los formatos de las respuestas a la pregunta "Me identifico"

for i in range(0, len(data)):
    data[i]['Me identifico'] = data[i]['Me identifico'].replace('ó','o')
    data[i]['Me identifico'] = data[i]['Me identifico'].capitalize()

# Creo un set con las respuestas unicas a la pregunta de "Me identifico"
resp_identidad_de_genero = set()
for i in range(0, len(data)):
    resp_identidad_de_genero.add(data[i]['Me identifico'])
resp_identidad_de_genero

#Limpio respuestas que no responden a lo preguntado
resp_identidad_de_genero.remove('Macho alfa')
resp_identidad_de_genero.remove('Helicoptero de combate')

#Agrupo respuestas que refieren al mismo genero
for i in range(0, len(data)):
    if data[i]['Me identifico'] != 'Varon cis' and data[i]['Me identifico'] != 'Varon trans':
        data[i]['Me identifico'] = data[i]['Me identifico'].replace('Varon','Varon cis')
        data[i]['Me identifico'] = data[i]['Me identifico'].replace('Hombre','Varon cis')
        data[i]['Me identifico'] = data[i]['Me identifico'].replace('Masculino','Varon cis')
    if data[i]['Me identifico'] != 'Mujer cis' and data[i]['Me identifico'] != 'Mujer trans':  
        data[i]['Me identifico'] = data[i]['Me identifico'].replace('Mujer','Mujer cis')
    data[i]['Me identifico'] = data[i]['Me identifico'].replace('Varon cis heterosexual','Varon cis')
    data[i]['Me identifico'] = data[i]['Me identifico'].replace('Varon cis ','Varon cis')

#Limpio las respuestas, eliminando aquellas que no responden lo preguntado, tomandolas como aquellas que fueron respondidas 2 veces como mucho.
identidad_de_genero = set()
for resp in resp_identidad_de_genero:
    contador = 0
    for i in range (0, len(data)):
        if data[i]['Me identifico'] == resp:
            contador += 1
    if contador > 2:
        identidad_de_genero.add(resp)

#Armo el diccionario de por genero
participantes_por_genero = []
for i in range(0,len(identidad_de_genero)):
    participantes_por_genero.append(0)
    
identidad_de_genero = list(identidad_de_genero)
dic_generos = dict(zip(identidad_de_genero, participantes_por_genero))

#Cuento la cantidad de participantes por genero
for i in range(0,len(data)):
    for genero in identidad_de_genero:
        if data[i]["Me identifico"] == genero:
            dic_generos[genero] += 1

#Base para porcentaje            
respuestas_genero = 0
for genero in identidad_de_genero:
    respuestas_genero += dic_generos[genero]

#Armo diciconario con porcentajes de participacion por genero
dic_participacion_generos = dic_generos

#Calculo participacion por genero
for genero in identidad_de_genero:
     dic_participacion_generos[genero]=round(dic_generos[genero]/respuestas_genero*100,2)

#Ordeno diccionario
sorted_dic_participacion_generos = ordenar_dict(dic_participacion_generos)

#Arreglo formato de porcentaje
sorted_dic_participacion_generos = formato_porcentaje(sorted_dic_participacion_generos)

#Calculo la mediana de salario por genero
mediana_salario_por_genero = mediana_salario_por_preg(identidad_de_genero,"Me identifico")



In [10]:
#Mediana salario por tecnologia

#Defino una funcion para el calculo de la mediana en caso de tecnologias
def mediana_salario_tecnologias(tecnologia, pregunta):
    respuestas_split = []
    for i in range(0, len(tecnologia)):
        respuestas_split.append(tecnologia[i].split(","))

    respuestas_totales = set()
    for respuesta in respuestas_split:
        for valor in respuesta:
            respuestas_totales.add(valor.strip().capitalize())

    #Limpieza de datos 
    respuestas_totales.discard('-')
    respuestas_totales.discard('--')
    respuestas_totales.discard('.')
    respuestas_totales.discard('0')
    respuestas_totales.discard('No')

    eliminables = set()
    for respuesta in respuestas_totales:
        if (respuesta[:3] == 'No ') or ('Ningun' in respuesta) or respuesta == 'n/a': 
            eliminables.add(respuesta)

    respuestas_totales = eliminables ^ respuestas_totales

    mediana_salario_por_tecnologia = {}
    for elemento in respuestas_totales:
        mediana_salario_por_tecnologia[elemento] = 0
    for elemento in respuestas_totales:
        salarios = []
        salarios_ordenado = []
        for i in range(0, len(data)):
            if data[i]['Salario en USD'] != '':
                if (elemento.lower() + ',') in (data[i][pregunta].lower() + ','):
                    salarios.append(data[i]['Salario en USD'])
        salarios_ordenado = sorted(salarios)
        if len(salarios)>1:
            if len(salarios)%2 != 0:
                indice_mediana = int(round(len(salarios)/2,0) - 1)
                mediana = salarios_ordenado[indice_mediana]
            else: 
                indice_1 = int(len(salarios)/2 - 1)
                indice_2 = int(len(salarios)/2)
                mediana = (salarios_ordenado[indice_1] + salarios_ordenado[indice_2])/2
        elif len(salarios) == 1:
            mediana = salarios_ordenado[0]
        else:
            mediana = 0
        mediana_salario_por_tecnologia[elemento] = mediana
    return ordenar_dict(mediana_salario_por_tecnologia)



In [11]:
#Primero las plataformas
plataformas = set()
for i in range(0, len(data)):
    if len(data[i]['Plataformas']) >0:
        plataformas.add(data[i]['Plataformas'])

plataformas = list(plataformas)

mediana_salario_por_plataforma = mediana_salario_tecnologias(plataformas, 'Plataformas')



In [28]:
def armado_dict_participacion_tecnologias(tecnologia, pregunta):
    respuestas_split = []
    for i in range(0, len(tecnologia)):
        respuestas_split.append(tecnologia[i].split(","))

    respuestas_totales = set()
    for respuesta in respuestas_split:
        for valor in respuesta:
            respuestas_totales.add(valor.strip().capitalize())

    #Limpieza de datos 
    respuestas_totales.discard('-')
    respuestas_totales.discard('--')
    respuestas_totales.discard('.')
    respuestas_totales.discard('0')
    respuestas_totales.discard('A')
    respuestas_totales.discard('No')
    respuestas_totales.discard('')

    eliminables = set()
    for respuesta in respuestas_totales:
        if (respuesta[:3] == 'No ') or ('Ningun' in respuesta) or respuesta == 'n/a': 
            eliminables.add(respuesta)

    respuestas_totales = eliminables ^ respuestas_totales
    
    #Armo el diccionario de cantidad de participantes por respuesta
    participantes_por_respuesta = []
    for i in range(0,len(respuestas_totales)):
        participantes_por_respuesta.append(0)

    respuestas_totales = list(respuestas_totales)
    dic_respuestas = dict(zip(respuestas_totales, participantes_por_respuesta))

    cant_respuestas = 0
    #Cuento la cantidad de participantes por respuesta unica
    for i in range(0,len(data)):
        for respuesta in respuestas_totales:
            if (respuesta.lower() + ',') in (data[i][pregunta].lower()+ ','):
                dic_respuestas[respuesta] += 1
                cant_respuestas += 1

    #Armo diccionario con porcentajes de participacion por respuesta unica
    dic_participacion = dic_respuestas

    #Calculo participacion en porcentaje
    for respuesta in respuestas_totales:
         dic_participacion[respuesta]=round(dic_respuestas[respuesta]/cant_respuestas*100,2)

    return dic_participacion

dic_participacion_plataformas = armado_dict_participacion_tecnologias(plataformas, 'Plataformas')
sorted_dic_participacion_plataformas = ordenar_dict(dic_participacion_plataformas)

sorted_dic_participacion_plataformas = formato_porcentaje(sorted_dic_participacion_plataformas)



In [13]:
#Identifico los distintos frameworks respondidos por los encuestados
frameworks_herram_lib = set()
for i in range(0, len(data)):
    if len(data[i]['Frameworks, herramientas y librerías'])>0:
        frameworks_herram_lib.add(data[i]['Frameworks, herramientas y librerías'])

frameworks_herram_lib = list(frameworks_herram_lib)

#Armo diccionario con participacion
dic_participacion_framework = armado_dict_participacion_tecnologias(frameworks_herram_lib, 'Frameworks, herramientas y librerías')

#Ordeno diccionario con participacion y acomodo formato de porcentaje
sorted_dic_participacion_framework = ordenar_dict(dic_participacion_framework)
sorted_dic_participacion_framework = formato_porcentaje(sorted_dic_participacion_framework)

#Calculo mediana de salario
mediana_salario_por_framework = mediana_salario_tecnologias(frameworks_herram_lib, 'Frameworks, herramientas y librerías')



In [14]:
bases_de_datos = set()
for i in range(0, len(data)):
    if len(data[i]['Bases de datos'])>0:
        bases_de_datos.add(data[i]['Bases de datos'])

bases_de_datos = list(bases_de_datos)

dic_participacion_por_BD = armado_dict_participacion_tecnologias(bases_de_datos, 'Bases de datos')
sorted_dic_participacion_BD = ordenar_dict(dic_participacion_por_BD)
sorted_dic_participacion_BD = formato_porcentaje(sorted_dic_participacion_BD)
mediana_salario_por_BD = mediana_salario_tecnologias(bases_de_datos, 'Bases de datos')




In [15]:
QA_testing = set()
for i in range(0, len(data)):
    QA_testing.add(data[i]['QA / Testing'])

QA_testing = list(QA_testing)

dic_participacion_por_QA_testing = armado_dict_participacion_tecnologias(QA_testing, 'QA / Testing')
sorted_dic_participacion_QA_testing = ordenar_dict(dic_participacion_por_QA_testing)
sorted_dic_participacion_QA_testing = formato_porcentaje(sorted_dic_participacion_QA_testing)
mediana_salario_por_QA_testing = mediana_salario_tecnologias(QA_testing, 'QA / Testing')




In [16]:
IDEs = set()
for i in range(0, len(data)):
    IDEs.add(data[i]['IDEs'])

IDEs = list(IDEs)

dic_participacion_por_IDEs = armado_dict_participacion_tecnologias(IDEs, 'IDEs')
sorted_dic_participacion_IDEs = ordenar_dict(dic_participacion_por_IDEs)
sorted_dic_participacion_IDEs = formato_porcentaje(sorted_dic_participacion_IDEs)
mediana_salario_por_IDEs = mediana_salario_tecnologias(IDEs, 'IDEs')




In [17]:
lenguajes = set()
for i in range(0, len(data)):
    lenguajes.add(data[i]['Lenguajes de programación o tecnologías.'])
lenguajes = list(lenguajes)

dic_participacion_por_lenguaje = armado_dict_participacion_tecnologias(lenguajes, 'Lenguajes de programación o tecnologías.')
sorted_dic_participacion_lenguaje = ordenar_dict(dic_participacion_por_lenguaje)
sorted_dic_participacion_lenguaje = formato_porcentaje(sorted_dic_participacion_lenguaje)
mediana_salario_por_lenguaje = mediana_salario_tecnologias(lenguajes, 'Lenguajes de programación o tecnologías.')




In [ ]:
#ANALISIS

In [18]:
print('Rol vs Sueldo')
print()
# Grafico porcentaje de participacion y mediana de salario por rol
print('Grafica de % de participacion y mediana del salario por rol.')
print()
graficar(sorted_dic_participacion_por_rol, 1, mediana_salario_por_rol)
print()

# Analizo los resultados
print('Observando el gráfico se puede observar que el 40.69% de los encuestados manifiestan trabajar como Developers. Esto era esperable dado que se trata de una encuesta enfocada a trabajadores en el área de la tecnología.')
print(f'Analizando la información de los salarios para los roles más frecuentes, podemos ver que el rol con una mediana de salario más alta es el de Manager/Director, con un salario de 2997 USD, mientras que el de mediana mas baja es el de HelpDesk, con una mediana de 716.0 USD')
print()
print(f'En total, los participantes contestaron un total de {len(roles)} roles distintos. Sin embargo, el 90.67% de las respuestas corresponden a únicamente 18 roles (aprox un 5% de los roles).')
print()

# Muestro los 10 roles mejor pagos y lo analizo
print('Top 10 roles mejor pagos')
print()
i=0
for rol in mediana_salario_por_rol:
    print (rol, end=" ")
    print (f'{mediana_salario_por_rol[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print(f'Si consideramos los {len(roles)} roles distintos, se obtuvo que el rol mejor pago es {rol_mejor_pago}, con una mediana de salario de {max(mediana_salario_por_rol.values())} USD')
print(f'En cuanto al rol peor pago, este fue el de "{rol_peor_pago}", con una mediana de salario de {min(mediana_salario_mayor_a_cero)} USD')

Rol vs Sueldo

Grafica de % de participacion y mediana del salario por rol.

Developer                 ----------------------------------------- 40.69%  1432.0 USD
SysAdmin / DevOps / SRE   ----------- 11.09%  1555.5 USD
Technical Leader          ------ 6.16%  2291.0 USD
QA / Tester               ---- 4.29%  1234.0 USD
Manager / Director        ---- 4.28%  2997.0 USD
Project Manager           --- 3.09%  2036.0 USD
HelpDesk                  --- 2.62%  716.0 USD
BI Analyst / Data Analyst -- 2.45%  1241.0 USD
Architect                 -- 2.23%  2195.0 USD
Consultant                -- 1.92%  1241.0 USD
Data Engineer             -- 1.76%  1756.0 USD
Business Analyst          -- 1.69%  1398.5 USD
Networking                -- 1.54%  1145.0 USD
UX                        -- 1.5%  1432.0 USD
Data Scientist            - 1.48%  1813.0 USD
Infosec                   - 1.43%  1581.5 USD
Recruiter / HR            - 1.24%  1026.0 USD
Product Manager           - 1.21%  2295.5 USD
Otros                  

In [19]:
print('Nivel de educacion vs Sueldos')
print()
print('Grafica de % de participacion y mediana del salario por nivel educativo.')
print()
graficar(sorted_dic_participacion_por_nivel, 0, mediana_salario_por_nivel)
print()

print('Porcentaje de respuestas por nivel educativo y por estado:')
print(lista_salida)
print()

print('Observando la participación por nivel educativo indicada en el gráfico, podemos ver que la mayor parte de los encuestados que respondieron esta pregunta son universitarios. Dentro de esta categoría, el 36.78% indicó haber terminado la carrera, mientras que un 26.06% no la completó y el restante 37.16% está cursando actualmente.')
print('El nivel educativo menos frecuente en las respuestas fue primaria, con tan solo 0.03% de los encuestados que respondieron la pregunta. En este caso, el 100% manifestó no haberla completado.')
print('Si ahora analizamos el estado de completitud de los estudios por nivel educativo, podemos notar que el Secundario es el nivel de mayor completitud entre quienes lo seleccionaron como nivel educativo, mientras que el menos completado es el primario.')
print('Al momento de analizar los salarios por nivel educativo, llama la atención que aquellos de nivel educativo más bajo (Primario) son los que tienen una mediana de salario más alta, y aquellos de nivel educativo más alto (Posdoctorado) son los de salario más bajo. Esto podría indicar que tal vez la mediana del salario no es muy representativa al momento de comparar los salarios de estos niveles, y también podría indicar que el tipo de cambio utilizado en algunos países no represente correctamente el salario percibido por los trabajadores.')
print('Excluyendo a estos dos niveles educativos, se puede ver que la mediana del salario aumenta a medida de que aumenta el nivel educativo, lo cual era esperable.')



Nivel de educacion vs Sueldos

Grafica de % de participacion y mediana del salario por nivel educativo.

Universitario -------------------------------------------------------------------- 68.5%  1575.0 USD
Terciario     ----------------- 17.3%  1336.0 USD
Posgrado      ------- 6.94%  1909.0 USD
Secundario    ------ 6.39%  1246.0 USD
Doctorado     - 0.72%  1857.0 USD
Posdoctorado   0.12%  573.0 USD
Primario       0.03%  2100.0 USD

Porcentaje de respuestas por nivel educativo y por estado:
[{'Universitario': {'Completado': '36.78%', 'Incompleto': '26.06%', 'En curso': '37.16%'}}, {'Secundario': {'Completado': '90.54%', 'Incompleto': '7.21%', 'En curso': '2.25%'}}, {'Terciario': {'Completado': '51.08%', 'Incompleto': '21.46%', 'En curso': '27.45%'}}, {'Posdoctorado': {'Completado': '50.0%', 'Incompleto': '25.0%', 'En curso': '25.0%'}}, {'Doctorado': {'Completado': '52.0%', 'Incompleto': '8.0%', 'En curso': '40.0%'}}, {'Primario': {'Completado': '0.0%', 'Incompleto': '100.0%', 'En curso':

In [20]:
print ('Genero vs Sueldos')
print()
print('Grafica de % de participacion y mediana del salario por genero.')
print()
graficar(sorted_dic_participacion_generos, 0, mediana_salario_por_genero)

print()
print('Observando la participación por genero indicada en el gráfico, podemos ver que la mayor parte de los encuestados (78.1%) que respondieron esta pregunta se identifican como hombres cis.')
print('Sin embargo, este grupo no es el que percibe el mayor salario. El grupo con una mediana de salario mas alta es el de Varon Trans en este caso, siendo este de 1766 USD.')
print('Se puede observar que quienes se identifican como mujeres (ya sea cis o trans) son quienes perciben el menor salario (el dato de salario para el grupo "Travesti" lo descarto porque no parece certero)')


Genero vs Sueldos

Grafica de % de participacion y mediana del salario por genero.

Varon cis         ------------------------------------------------------------------------------ 78.1%  1604.0 USD
Mujer cis         ----------------- 16.59%  1229.0 USD
Prefiero no decir ---- 4.19%  1527.0 USD
No binarie        - 0.74%  1319.0 USD
Varon trans        0.21%  1766.0 USD
Mujer trans        0.1%  954.0 USD
Travesti           0.07%  1.0 USD

Observando la participación por genero indicada en el gráfico, podemos ver que la mayor parte de los encuestados (78.1%) que respondieron esta pregunta se identifican como hombres cis.
Sin embargo, este grupo no es el que percibe el mayor salario. El grupo con una mediana de salario mas alta es el de Varon Trans en este caso, siendo este de 1766 USD.
Se puede observar que quienes se identifican como mujeres (ya sea cis o trans) son quienes perciben el menor salario (el dato de salario para el grupo "Travesti" lo descarto porque no parece certero)


In [21]:
print('Tecnologias populares')
print()
print('Para analizar las tecnologias populares, decidi hacerlo por grupo y no mezclar las distintas tecnologias (Plataformas, Frameworks, IDEs, etc.)')


Tecnologias populares

Para analizar las tecnologias populares, decidi hacerlo por grupo y no mezclar las distintas tecnologias (Plataformas, Frameworks, IDEs, etc.)


In [22]:
#Grafico plataformas
print('Grafico de participacion y mediana del salario por Plataforma')
print()
graficar(sorted_dic_participacion_plataformas, 1, mediana_salario_por_plataforma)

print()
print('Se presentan en el grafico aquellas plataformas respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la plataforma mas utilizada por los encuestados es Linux, con un 18.52% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.')
print()

# Muestro las 10 plataformas asociadas a mejores salarios y lo analizo
print('Top 10 plataformas con mediana de salario mas alta:')
print()
i=0
for rol in mediana_salario_por_plataforma:
    print (rol, end=" ")
    print (f'{mediana_salario_por_plataforma[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print('Descartando el primer dato, se obtiene que la plataforma asociada a una mediana de salario mas alta es Rabbitmq, siendo esta de 152984 USD')




Grafico de participacion y mediana del salario por Plataforma

Linux                            ------------------- 18.52%  1623.0 USD
Docker                           --------------- 15.39%  1909.0 USD
Amazon web services              ------------- 12.79%  2019.0 USD
Windows server                   ---------- 9.71%  1336.0 USD
Azure                            ------- 7.03%  1623.0 USD
Kubernetes                       ------- 6.7%  2291.0 USD
Vmware                           ----- 5.36%  1432.0 USD
Google cloud platform/app engine ----- 4.73%  1856.0 USD
Firebase                         --- 2.86%  1632.5 USD
Serverless                       --- 2.74%  2291.0 USD
Openshift                        -- 2.1%  2004.0 USD
Sap                              -- 2.04%  1432.0 USD
Heroku                           - 1.01%  1765.0 USD
Otros                            -------- 9.02%

Se presentan en el grafico aquellas plataformas respondidas por mas de un 1% de los encuestados. De su observacion se d

In [23]:
#Grafico participacion y mediana de salario por tecnologia de framework
print('Grafico de participacion y mediana del salario por Frameworks, herramientas y librerias')
print()
graficar(sorted_dic_participacion_framework, 1, mediana_salario_por_framework)

print()
print('Se presentan en el grafico aquellas Frameworks, herramientas y librerias respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la mas utilizada por los encuestados es Node.js, con un 12.78% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.')
print()

# Muestro las 10 frameworks, herramientas y liberias asociadas a mejores salarios y lo analizo
print('Top 10 Frameworks, herramientas y librerias con mediana de salario mas alta:')
print()
i=0
for rol in mediana_salario_por_framework:
    print (rol, end=" ")
    print (f'{mediana_salario_por_framework[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print('Observando los 10 Frameworks, herramientas y librerias con mediana de salario mas alta, se obtiene que la de mayor mediana de salario es ".net full framework", con una mediana de 94219 USD')

Grafico de participacion y mediana del salario por Frameworks, herramientas y librerias

Node.js    ------------- 12.78%  1814.0 USD
React.js   ------------ 12.1%  1813.0 USD
Bootstrap  ---------- 9.7%  1336.0 USD
Jquery     -------- 8.07%  1384.0 USD
Angular    -------- 7.75%  1594.0 USD
.net core  ------- 6.54%  1613.0 USD
Spring     ------ 6.39%  1913.5 USD
Hibernate  ---- 3.8%  1907.0 USD
Wordpress  ---- 3.78%  1145.0 USD
Vue.js     --- 3.32%  1360.0 USD
Flask      --- 2.9%  2004.0 USD
Django     --- 2.68%  1718.0 USD
Laravel    --- 2.51%  1317.0 USD
Spark      -- 2.03%  2019.5 USD
Sap        -- 1.88%  1422.0 USD
Symfony    - 1.3%  1394.0 USD
Rails      - 1.24%  1940.5 USD
Hadoop     - 1.11%  1902.5 USD
Tensorflow - 1.04%  2100.0 USD
Otros      -------- 9.08%

Se presentan en el grafico aquellas Frameworks, herramientas y librerias respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la mas utilizada por los encuestados es Node.js, con un 12.78% de re

In [24]:
#Grafico participacion y mediana de salario por tecnologia de framework
print('Grafico de participacion y mediana del salario por Base de Datos')
print()
graficar(sorted_dic_participacion_BD, 1, mediana_salario_por_BD)


print()
print('Se presentan en el grafico aquellas Bases de Datos respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la mas utilizada por los encuestados es SQL, con un 17.06% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.')
print()

# Muestro las 10 plataformas asociadas a mejores salarios y lo analizo
print('Top 10 Bases de Datos con mediana de salario mas alta:')
print()
i=0
for rol in mediana_salario_por_BD:
    print (rol, end=" ")
    print (f'{mediana_salario_por_BD[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print('Observando las 10 Bases de Datos con mediana de salario mas alta, y descartando el primer dato que parece no ser real, se obtiene que la de mayor mediana de salario es "Amazon documentdb", con una mediana de 152984.0 USD')

Grafico de participacion y mediana del salario por Base de Datos

Sql                     ----------------- 17.06%  1575.0 USD
Mysql                   ----------- 11.49%  1527.0 USD
Sql server              --------- 8.92%  1432.0 USD
Microsoft sql server    --------- 8.89%  1432.0 USD
Postgresql              ------- 7.4%  1813.0 USD
Mongodb                 ----- 5.45%  1959.0 USD
Oracle                  ----- 4.51%  1608.0 USD
Mariadb                 ---- 4.19%  1527.0 USD
Redis                   --- 3.34%  2305.0 USD
Elasticsearch           --- 3.18%  2291.0 USD
Dynamodb                --- 2.68%  2291.0 USD
Amazon dynamodb         --- 2.67%  2291.0 USD
Amazon rds/aurora       -- 2.24%  2243.0 USD
Microsoft azure (tables -- 2.23%  1651.0 USD
Etc)                    -- 2.23%  1651.0 USD
Cosmosdb                -- 2.23%  1651.0 USD
Sqlite                  -- 1.97%  1558.0 USD
Google cloud storage    - 1.46%  1856.5 USD
Amazon redshift         - 1.07%  2195.0 USD
Otros                   -

In [25]:
#Grafico participacion y mediana de salario por tecnologia de QA/Testing
print('Grafico de participacion y mediana del salario por tecnologia de QA/Testing')
print()
graficar(sorted_dic_participacion_QA_testing, 1, mediana_salario_por_QA_testing)


print()
print('Se presentan en el grafico aquellas tecnologias de QA/Testing respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la mas utilizada por los encuestados es Postman, con un 20.47% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.')
print()

# Muestro las 10 tecnologias de QA/Testing asociadas a mejores salarios y lo analizo
print('Top 10 tecnologias de QA/Testing con mediana de salario mas alta:')
print()
i=0
for rol in mediana_salario_por_QA_testing:
    print (rol, end=" ")
    print (f'{mediana_salario_por_QA_testing[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print('Observando las 10 tecnologias de QA/Testing con mediana de salario mas alta, se obtiene que la de mayor mediana de salario es "Jasmine", con una mediana de 139158.0 USD')

Grafico de participacion y mediana del salario por tecnologia de QA/Testing

Postman                -------------------- 20.47%  1689.0 USD
Junit                  ------------- 12.85%  2023.0 USD
Jest                   ---------- 10.05%  2032.0 USD
M                      -------- 8.26%  1909.0 USD
Selenium               ------- 7.39%  1909.0 USD
Soapui                 ----- 5.37%  1623.0 USD
Cypress                ----- 4.97%  2338.0 USD
Mocha                  --- 3.43%  1950.0 USD
Cucumber               --- 3.41%  2014.0 USD
Phpunit                --- 3.05%  1527.0 USD
Visual studio coded ui -- 2.22%  1432.0 USD
Tfs                    -- 1.79%  1464.5 USD
Appium                 -- 1.66%  1909.0 USD
Nunit                  -- 1.64%  1718.0 USD
Test                   - 1.44%  1859.5 USD
Karma                  - 1.28%  1909.0 USD
Puppeteer              - 1.13%  2434.0 USD
Otros                  ------------ 9.59%

Se presentan en el grafico aquellas tecnologias de QA/Testing respondidas p

In [26]:
#Grafico participacion y mediana de salario por IDE
print('Grafico de participacion y mediana del salario por IDE')
print()
graficar(sorted_dic_participacion_IDEs, 1, mediana_salario_por_IDEs)

print()
print('Se presentan en el grafico aquellas IDEs respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la mas utilizada por los encuestados es Code, con un 23.73% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.')
print()

# Muestro las 10 IDEs asociadas a mejores salarios y lo analizo
print('Top 10 IDEs con mediana de salario mas alta:')
print()
i=0
for rol in mediana_salario_por_IDEs:
    print (rol, end=" ")
    print (f'{mediana_salario_por_IDEs[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print('Observando las 10 IDEs con mediana de salario mas alta, se obtiene que la de mayor mediana de salario es "Oracle forms & reports", con una mediana de 108933.0 USD')

Grafico de participacion y mediana del salario por IDE

Code               ------------------------ 23.73%  1594.0 USD
Visual studio code ----------------------- 23.28%  1594.0 USD
Intellij           ------- 6.77%  2062.0 USD
Visual studio      ------- 6.53%  1522.5 USD
Notepad++          ------ 5.77%  1489.0 USD
Vim                ----- 5.34%  1909.0 USD
Sublime text       ----- 4.69%  1623.0 USD
Eclipse            ---- 3.74%  1575.0 USD
Android studio     --- 2.97%  1642.5 USD
Vi                 --- 2.56%  1909.0 USD
Pycharm            --- 2.52%  1813.0 USD
Ipython / jupyter  -- 1.91%  1732.0 USD
Xcode              -- 1.5%  2052.0 USD
Atom               -- 1.5%  1604.0 USD
Nano               - 1.31%  1432.0 USD
Webstorm           - 1.2%  1937.0 USD
Phpstorm           - 1.17%  1527.0 USD
Netbeans           - 1.16%  1336.0 USD
Otros               2.35%

Se presentan en el grafico aquellas IDEs respondidas por mas de un 1% de los encuestados. De su observacion se desprende que la mas ut

In [27]:
#Grafico participacion y mediana de salario por lenguaje de programacion
print('Grafico de participacion y mediana del salario por Lenguaje de Programacion')
print()
graficar(sorted_dic_participacion_lenguaje, 1, mediana_salario_por_lenguaje)

print()
print('Se presentan en el grafico aquellos lenguajes de programacion respondidos por mas de un 1% de los encuestados. De su observacion se desprende que el mas utilizado por los encuestados es Javascript, con un 14.7% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.')
print()

# Muestro las 10 IDEs asociadas a mejores salarios y lo analizo
print('Top 10 lenguajes con mediana de salario mas alta:')
print()
i=0
for rol in mediana_salario_por_lenguaje:
    print (rol, end=" ")
    print (f'{mediana_salario_por_lenguaje[rol]} USD')
    i+=1 
    if i==10:
        break
print()

print('Observando los 10 lenguajes con mediana de salario mas alta, y descartando el primer valor que parece no ser real, se obtiene que el de mayor mediana de salario es "Mpi", con una mediana de 91574.0 USD')

Grafico de participacion y mediana del salario por Lenguaje de Programacion

Javascript --------------- 14.7%  1563.0 USD
Sql        ------------- 12.83%  1470.0 USD
Html       ---------- 10.39%  1415.0 USD
Python     --------- 9.16%  1752.5 USD
Java       -------- 7.53%  1813.0 USD
Css        ------- 6.94%  1394.0 USD
Bash/shell ------ 5.71%  1794.5 USD
Typescript ----- 5.36%  1718.0 USD
Php        ----- 4.61%  1336.0 USD
.net       ---- 4.27%  1527.0 USD
C#         ---- 3.7%  1527.0 USD
Go         -- 2.27%  2291.0 USD
Kotlin     - 1.28%  2291.0 USD
C          - 1.13%  1623.0 USD
Vba        - 1.01%  1432.0 USD
C++        - 1.0%  1527.0 USD
Otros      -------- 8.11%

Se presentan en el grafico aquellos lenguajes de programacion respondidos por mas de un 1% de los encuestados. De su observacion se desprende que el mas utilizado por los encuestados es Javascript, con un 14.7% de respuestas. Sin embargo, la mediana del salario no es la mayor en este caso.

Top 10 lenguajes con mediana de 